In [1]:
import numpy as np # linear algebrae
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression

import os

# from polire import IDW

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from datetime import date, timedelta
import datetime

import json
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss
from PIL import Image
import torchvision.transforms as transforms

from tqdm import tqdm

In [2]:
class CFG:
    evaluation_time_gap = 1
    convert_numpy = False
    target_list = ['pm2_5', 'pm10']
    features = ['timeOfDay', 'lat', 'lon', 'distance', 'bus_count', 'day_of_week', 'pm2_5', 'pm10']
    img_cols = ['lat_i', 'lon_j']

    batch_size = 128
    num_epochs = 50
    day_len = 35

    # model_size = 'vl'
    model_dropout = 0.2

    hidden_dim = 128
    num_layers = 6
    
    lr = 3e-3
    patience=2
    factor=0.9

    model_kind="gru"
    bidirectional=False

    pm2_5_thresholds = [0, 30, 60, 90, 120, 250, 2000]
    pm10_thresholds = [0, 50, 100, 250, 350, 430, 2000]
    aqi_category = ["Good", "Satisfactory", "Moderate", "Poor", "Very Poor", "Severe"]
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset

In [3]:
class SequenceImageDataset(Dataset):

    def __init__(self, df, img_list, 
                 features = None, img_cols = None, sequence_length=70,
                 forecast_horizon=35):
        
        features = CFG.features if features is None else features
        img_cols = CFG.img_cols if img_cols is None else img_cols
        self.features = features
        self.img_cols = img_cols
        
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon

        self.split_df(df)
        
        self.img_list = img_list
        self.proc_img(img_list)

    def split_df(self, df):
        df = df.sort_values(['lat', 'lon', 'date_value', 'timeOfDay'])
        lat_lon_pairs = df.groupby(['lat', 'lon'])['pm2_5'].count().reset_index()[['lat', 'lon']].values

        self.sequences = []

        for i in lat_lon_pairs:
            lat_lon_values = df[(df['lat'] == i[0]) & (df['lon'] == i[1])]
            self.sequences.append(lat_lon_values)

        self.X_seq = []
        self.y_seq = []
        self.masked_seq = []
        self.img_id = []

        self.total_len = 0
        
        for s in self.sequences:
            s = s.sort_values(['date_value', 'timeOfDay'])
            self.X_seq.append(np.array(s[self.features].values, dtype=np.float32))
            self.y_seq.append(np.array(s[CFG.target_list].values, dtype=np.float32))
            
            self.masked_seq.append(1 - np.array(s['missing'].values, dtype=np.int8))
            # self.masked_seq.append(np.array(s['missing'].values, dtype=np.int8))
            
            self.img_id.append(s[self.img_cols].values)
            
            curr_len = len(self.y_seq[-1])
            curr_len -= self.sequence_length  + self.forecast_horizon - CFG.day_len
            self.total_len += curr_len // CFG.day_len
            self.per_len = curr_len // CFG.day_len
    
    def __len__(self):
        return self.total_len

    def proc_img(self, img_list):
        self.imgs = {}

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        for (i, j), img_file in img_list:
            img = Image.open(img_file).convert("RGB")
            tensor_img = transform(img)
            self.imgs[(i, j)] = tensor_img

    def get_index(self, i):
        lat_lon = i // self.per_len
        within_idx = i % self.per_len
        return lat_lon, within_idx
    
    def __getitem__(self, i):
        lat_lon, within_idx = self.get_index(i)

        within_idx *= CFG.day_len
        start = within_idx
        X_end = within_idx + self.sequence_length
        y_end = X_end + self.forecast_horizon
        
        
        X = torch.tensor(self.X_seq[lat_lon][start:X_end]).to(device)
        y = torch.tensor(self.y_seq[lat_lon][X_end:y_end]).to(device)
        masks = torch.tensor(self.masked_seq[lat_lon][X_end:y_end] # , dtype=torch.int32
                            ).to(device)
        masks = torch.reshape(masks, (-1, 1))

        img = self.imgs[tuple(self.img_id[lat_lon][start])]
        
        return X, y, masks, img

In [4]:
df = pd.read_csv('/kaggle/input/airdelhi-baselines-deepengineering/dense_df.csv')

df = df.drop(columns = 'Unnamed: 0')
df['date_value'] = pd.to_datetime(df['date_value'])

dates = pd.to_datetime([df['date_value'].min(), df['date_value'].max()])

max_train_date = dates.min() + (dates.max() - dates.min()) * 0.75
max_train_date = max_train_date.floor("D")
min_train_date = df['date_value'].min()
max_date = df['date_value'].max().floor("D")

metrics_dict = {
    'MSE': mean_squared_error, 
    'r2 score': r2_score, 
    'MAE': mean_absolute_error,
}

target_list = CFG.target_list

features = ['date_value', 'timeOfDay', 'lat', 'lon', 'day_of_week', 'distance', 'bus_count']

CFG.base_features = ['timeOfDay', 'lat', 'lon', 'day_of_week', 'distance', 'bus_count']
CFG.features = CFG.base_features

min_train_date, max_train_date

(Timestamp('2020-11-01 00:00:00'), Timestamp('2021-01-07 00:00:00'))

In [5]:
def add_lag_features(df, lags = [1]):

    df = df.copy()
    
    df.sort_values(by=["lat", "lon", "timeOfDay", "date_value"], inplace=True)

    added_features = []
    for l in lags:
        df[f'pm2_5_lag_{l}'] = df.groupby(
            ['timeOfDay', 'lat', 'lon'])['pm2_5'].shift(l)
        df[f'pm10_lag_{l}'] = df.groupby(
            ['timeOfDay', 'lat', 'lon'])['pm10'].shift(l)

        added_features.append(f'pm2_5_lag_{l}')
        added_features.append(f'pm10_lag_{l}')

        df.sort_values(by=["lat", "lon", "date_value"], inplace=True)

    # Group by latitude and longitude
    grouped = df.groupby(["lat", "lon"])

    # Function to fill NaN values based on previous mean
    def fill_na_with_previous_mean(group):
        for col in group.columns:
            if col not in ["date_value", "lat", "lon"]:
                group[col] = group[col].astype(float)  # Ensure numeric columns
                group[col] = group[col].fillna(group[col].expanding().mean().shift())  # Previous days' mean
                
                # If still NaN (first row), replace with overall mean
                overall_mean = df[col].mean(skipna=True)
                group[col] = group[col].fillna(overall_mean)
        return group

    # Apply the function to each group
    df = grouped.apply(fill_na_with_previous_mean)
    df.reset_index(drop=True, inplace=True)
    df = df.sort_values(by = ['date_value', 'timeOfDay', 'lat', 'lon'])
    df.reset_index(drop=True, inplace=True)
    CFG.features += added_features
    return df
    
df = add_lag_features(df, lags = [7])

<ipython-input-5-a10b91631712>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = grouped.apply(fill_na_with_previous_mean)


In [6]:
df = df.sort_values(by=['lat', 'lon', 'timeOfDay', 'date_value'])

In [7]:
class CustomScaler:

    def __init__(self, df):
        self.pm2_5_max = float(df['pm2_5'].max())
        self.pm2_5_min = float(df['pm2_5'].min())
        
        self.pm10_max = float(df['pm10'].max())
        self.pm10_min = float(df['pm10'].min())

    def transform(self, df):
        df = df.copy()
        df['pm2_5'] = (df['pm2_5'] - self.pm2_5_min) / (self.pm2_5_max - self.pm2_5_min)
        df['pm10'] = (df['pm10'] - self.pm10_min) / (self.pm10_max - self.pm10_min)
        return df

    def inverse_transform(self, X):
        X[:, 0] = X[:, 0] * (self.pm2_5_max - self.pm2_5_min) + self.pm2_5_min
        X[:, 1] = X[:, 1] * (self.pm10_max - self.pm10_min) + self.pm10_min
        return X

target_scaler = CustomScaler(df)
df = target_scaler.transform(df)

CFG.features = ['timeOfDay', 'lat', 'lon', 'distance', 'bus_count', 
                'day_of_week', 'pm2_5', 'pm10', 'pm2_5_lag_7', 'pm10_lag_7']

df['day_of_week'] = df['date_value'].dt.dayofweek

scaler = MinMaxScaler()
df[CFG.features] = scaler.fit_transform(df[CFG.features])

In [8]:
with open("/kaggle/input/delhi-satellite-images-grid/tile_metadata.json", 'r') as f:
    metadata = json.load(f)

In [9]:
def process_metadata(metadata):
    img_list = []
    for k, v in metadata.items():
        i = v['row']
        j = v['col']
        img_list.append([(i, j), 
            f"/kaggle/input/delhi-satellite-images-grid/tiles_32x32/tile_{i}_{j}.png"])

    return img_list

img_list = process_metadata(metadata)

In [10]:
lat = df['lat'].unique()
lon = df['lon'].unique()

lat = np.sort(lat)
lon = np.sort(lon)

min_dist_lat = 1000
for i in range(len(lat) - 1):
    min_dist_lat = min(min_dist_lat, lat[i+1] - lat[i])
print(min_dist_lat)

for i in range(len(lat)):
    assert (lat[i] / min_dist_lat - round(lat[i] / min_dist_lat)) < 1e-6

min_dist_lon = 1000
for i in range(len(lon) - 1):
    min_dist_lon = min(min_dist_lon, lon[i+1] - lon[i])
print(min_dist_lon)

for i in range(len(lon)):
    assert (lon[i] / min_dist_lon - round(lon[i] / min_dist_lon)) < 1e-6

0.04999999999997419
0.05263157894735837


In [11]:
def rounded_lat_lon(df):
    df = df.copy()
    df['lat_i'] = np.round(df['lat'] / min_dist_lat) + 2
    df['lon_j'] = np.round(df['lon'] / min_dist_lon) + 4
    return df

df = rounded_lat_lon(df)

In [12]:
train_ds = SequenceImageDataset(df[df['date_value'] <= max_train_date], img_list)
test_ds = SequenceImageDataset(df[df['date_value'] > max_train_date], img_list)

train_dl = DataLoader(train_ds, batch_size = CFG.batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size = CFG.batch_size, shuffle=True)

In [13]:
xt, yt, mt, i = train_ds[0]
xt, yt, mt, i2 = train_ds[1]
xt.shape, yt.shape, mt.shape, i.shape

(torch.Size([70, 10]),
 torch.Size([35, 2]),
 torch.Size([35, 1]),
 torch.Size([3, 32, 32]))

In [14]:
assert torch.all(i == i2)

## RNN / GRU

In [15]:
class ImageMultiStepPredictor(nn.Module):
    def __init__(
        self, 
        input_size, 
        hidden_size, 
        num_layers, 
        dropout=0.2,
        output_size=2, 
        forecast_horizon=35,
        model_kind=None,
    ):
        """
        RNN-based model for multi-step time-series forecasting.

        Also takes in a satellite image representing the area being forecasted over

        Args:
            input_size (int): Number of input features.
            hidden_size (int): Hidden state size in RNN.
            num_layers (int): Number of stacked RNN layers.
            output_size (int): Number of target variables.
            forecast_horizon (int): Number of future time-steps to predict.
        """
        super(ImageMultiStepPredictor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.forecast_horizon = forecast_horizon
        if model_kind is None:
            model_kind = CFG.model_kind
        
        if model_kind == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        elif model_kind == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, 
                               batch_first=True, dropout=dropout,
                               bidirectional = CFG.bidirectional)
        elif model_kind == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, 
                              batch_first=True, dropout=dropout,
                              bidirectional = CFG.bidirectional)
        
        self.rnn = self.rnn.to(device)
        # self.conv_model = nn.Sequential(
        #     nn.Conv2d(3, 32, 3), # 30x30
        #     nn.ReLU(),
        #     nn.MaxPool2d(2), # 15x15
        #     nn.Conv2d(32, 64, 4), # 12x12
        #     nn.BatchNorm2d(64),
        #     nn.Dropout(dropout),
        #     nn.ReLU(),
        #     nn.MaxPool2d(2), # 6x6
        #     nn.Conv2d(64, 64, 3), # 4x4
        #     nn.BatchNorm2d(64),
        #     nn.ReLU(),
        #     nn.Dropout(dropout),
        #     nn.MaxPool2d(2), # 2x2
        #     nn.Flatten(),
        #     nn.Linear(4*64, 32),
        #     nn.Dropout(dropout),
        #     nn.ReLU()
        # )

        self.conv_model = nn.Sequential(
            nn.Conv2d(3, 16, 3), # 30x30
            nn.ReLU(),
            nn.MaxPool2d(2), # 15x15
            nn.Conv2d(16, 32, 4), # 12x12
            nn.BatchNorm2d(32),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool2d(2), # 6x6
            nn.Conv2d(32, 32, 3), # 4x4
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.MaxPool2d(2), # 2x2
            nn.Flatten(),
            nn.Linear(4*32, 32),
            nn.Dropout(dropout),
            nn.ReLU()
        ).to(device)
        
        # Fully connected layer maps hidden state to output
        self.fc = nn.Linear(hidden_size + 32, forecast_horizon * output_size).to(device)

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if param.dim() < 2:
                nn.init.zeros_(param)
            elif 'weight_ih' in name:
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)
            elif 'fc' in name and 'weight' in name:
                nn.init.kaiming_uniform_(param, nonlinearity='relu')
            elif 'conv' in name and 'weight' in name:
                nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')


    def forward(self, x, img):
        batch_size = x.size(0)
        
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)  # (batch_size, sequence_length, hidden_size)
        last_hidden_state = out[:, -1, :]  # (batch_size, hidden_size)

        features = self.conv_model(img)

        h = torch.cat([last_hidden_state, features], dim=1)

        out = self.fc(h)  # (batch_size, forecast_horizon * output_size)

        # Reshape to (batch_size, forecast_horizon, output_size)
        out = out.view(batch_size, self.forecast_horizon, -1)
        return out

def get_optimizer(model, lr = 1e-4, weight_decay = 1e-5):
    optimizer = Adam(params = model.parameters(), 
                     lr = lr, weight_decay = weight_decay)
    return optimizer

def get_reduce_lr(optimizer, factor=0.1, patience=2):
    scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = factor, patience = patience)
    return scheduler

In [16]:
def get_model(input_size = 10, hidden_size = 32, num_layers = 3, forecast_horizon=35,
             model_kind = 'gru'):
    if input_size is None:
        input_size = len(CFG.features)
    model = ImageMultiStepPredictor(input_size, hidden_size, num_layers, 
                               output_size=2, forecast_horizon=forecast_horizon,
                                   dropout=CFG.model_dropout, model_kind=model_kind)
    return model

## Train and Validation Functions

In [17]:
class MaskedMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedMSELoss, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        loss = (preds - targets) ** 2
        masked_loss = (loss * mask).sum() / (mask.sum() + self.eps)
        return masked_loss

class MaskedMAELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedMAELoss, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        loss = abs(preds - targets)
        masked_loss = (loss * mask).sum() / (mask.sum() + self.eps)
        return masked_loss

class MaskedR2(nn.Module):
    def __init__(self, eps=1e-6):
        super(MaskedR2, self).__init__()
        self.eps = eps

    def forward(self, preds, targets, mask):
        error = ((targets - preds) ** 2) * mask
        masked_mse = error.sum() / (mask.sum() + self.eps)
        mean_y = (targets * mask).sum() / (mask.sum() + self.eps)
        total_var = (((targets - mean_y) ** 2) * mask).sum() / (mask.sum() + self.eps)
        r2 = 1 - (masked_mse / (total_var + self.eps))
        return r2

masked_metrics_dict = {
    'MSE': MaskedMSELoss(),
    'MAE': MaskedMAELoss(),
    'r2 score': MaskedR2(),
}

In [18]:
class RunningLoss:
    def __init__(self, window = 10):
        self.loss = 0
        self.total = 0
        self.loss_last = []
        self.total_last = []

        self.window = window
    
    def update(self, loss, batch_size):
        total = 1
        self.loss += loss
        self.total += 1
        
        self.loss_last.append(loss)
        self.total_last.append(total)
        if len(self.loss_last) > self.window:
            self.loss_last.pop(0)
            self.total_last.pop(0)
    
    def reset(self):
        self.loss = 0
        self.total = 0
        self.loss_last = []
        self.total_last = []

    def print(self):
        print(f"Accuracy: {self.loss / self.total}")
    
    def get_curr_stats(self):
        return sum(self.loss_last) / sum(self.total_last)

    def total_stats(self):
        return self.loss / self.total


In [19]:
def validate(
    model,
    test_dl,
    criterion,
    verbose=True,
    get_predict=False,
):
    model.eval()
    running_loss = RunningLoss()

    predict_array = []
    true_array = []
    mask_array = []
    
    if verbose >= 1:
        pbar = tqdm(total = len(test_dl), ncols = 110, desc = "Validation Progress") # , dynamic_ncols=True, leave=False)
    else:
        pbar = None

    with torch.no_grad():
        for i, (x, y, mask, img) in enumerate(test_dl):
            x, y, mask, img = x.to(device), y.to(device), mask.to(device), img.to(device)
            pred = model(x, img)

            pred = target_scaler.inverse_transform(pred)
            y = target_scaler.inverse_transform(y)

            temp_loss = criterion(pred, y, mask)
            running_loss.update(temp_loss.item(), batch_size=x.shape[0])

            if get_predict:
                predict_array.append(pred)
                true_array.append(y)
                mask_array.append(mask)

            if verbose >= 1:
                total = len(test_dl)
                curr_loss = np.sqrt(running_loss.get_curr_stats())
                pbar.set_description(f"Validation Step {i} / {total}")
                pbar.set_postfix(Loss=f"{curr_loss:.4f}")
                pbar.update(1)
        if verbose >= 1:
            pbar.close()

    model.train()
    
    if get_predict:
        return predict_array, true_array, mask_array, np.sqrt(running_loss.total_stats())
    
    return np.sqrt(running_loss.total_stats())


In [20]:
def train(
        model,
        criterion,
        optimizer,
        num_epochs,
        train_dataloader,
        val_dataloader,
        scheduler=None,
        do_validate=False,
        validate_frequency=1,
        verbose=True,
        metrics_dict=None,
        model_name = ""
):
    if metrics_dict is None:
        metrics_dict = masked_metrics_dict
    
    loss_list = []
    val_loss_list = []
    all_loss_list = []

    best_val_loss = 100000000
    best_val_loss25 = 100000000
    best_val_loss10 = 100000000

    for epoch in tqdm(range(num_epochs), disable=verbose >= -1):
        running_loss = RunningLoss()
        if verbose >= 1:
            pbar = tqdm(total = len(train_dataloader), ncols = 110, desc = "Training Progress") # , dynamic_ncols=True, leave=False)
        else:
            pbar = None
        for i, (x, y, mask, img) in enumerate(train_dataloader):
            x, y, mask, img = x.to(device), y.to(device), mask.to(device), img.to(device)

            optimizer.zero_grad()
            
            pred = model(x, img)
            loss = criterion(pred, y, mask)
            loss.backward()

            optimizer.step()

            all_loss_list.append(loss.item())

            with torch.no_grad():
                pred = target_scaler.inverse_transform(pred)
                y = target_scaler.inverse_transform(y)
                temp_loss = torch.sqrt(criterion(pred, y, mask))
                running_loss.update(temp_loss.item(), batch_size=x.shape[0])

            if verbose >= 1:
                total = len(train_dataloader)
                curr_loss = running_loss.get_curr_stats()
                pbar.set_description(f"Epoch {epoch}: Step {i} / {total}")
                pbar.set_postfix(Loss=f"{curr_loss:.4f}")
                pbar.update(1)
        if verbose >= 1:
            print(f"Train Loss Total: {running_loss.total_stats()}")
            pbar.close()
        
        if do_validate and epoch % validate_frequency == validate_frequency - 1:
            pred1, y1, mask1, val_loss = validate(model, val_dataloader, criterion, 
                                           verbose=verbose, get_predict=True)
            pred1 = torch.concat(pred1)
            y1 = torch.concat(y1)
            mask1 = torch.concat(mask1)
            
            val_loss_list.append(val_loss)
            if scheduler is not None:
                scheduler.step(val_loss)
            
            if verbose >= 1:
                print(f"Val Loss Total: {val_loss}")

            if val_loss < best_val_loss:
                if verbose >= 0:
                    print(f"Better Val Loss: {val_loss} < {best_val_loss}")
                best_val_loss = val_loss
                torch.save(model.state_dict(), f"./{model_name}best.pth")
            
            for i, t in enumerate(CFG.target_list):
                for n, m in metrics_dict.items():
                    if n == 'MSE':
                        # print(y1[:, :, i].shape, pred1[:, :, i].shape, mask1.shape)
                        mse_loss = np.sqrt(m(pred1[:, :, i], y1[:, :, i], mask1[:, :, 0]).cpu().numpy())
                        if verbose >= 1:
                            print(f"{t}: {n}: {mse_loss}")
                        
                        if i == 0 and mse_loss < best_val_loss25:
                            if verbose >= 0:
                                print(f"----------Better MSE on pm2_5 {mse_loss} < {best_val_loss25}")
                            torch.save(model.state_dict(), f"./{model_name}2_5.pth")
                            best_val_loss25 = mse_loss
                        if i == 1 and mse_loss < best_val_loss10:
                            if verbose >= 0:
                                print(f"----------Better MSE on pm10 {mse_loss} < {best_val_loss10}")
                            torch.save(model.state_dict(), f"./{model_name}10.pth")
                            best_val_loss10 = mse_loss
                    
                    if verbose >= 1:
                        if n == 'MSE':
                            mse_loss = np.sqrt(m(pred1[:, :, i], y1[:, :, i], mask1[:, :, 0]).cpu().numpy())
                            print(f"{t}: {n}: {mse_loss}")
                        else:
                            print(f"{t}: {n}: {(m(pred1[:, :, i], y1[:, :, i], mask1[:, :, 0]))}")
        
        epoch_loss = running_loss.total_stats()
        loss_list.append(epoch_loss)
    
    return loss_list, val_loss_list, all_loss_list


## Run

In [21]:
# model = get_model(hidden_size = CFG.hidden_dim, num_layers = CFG.num_layers).to(device)
# criterion = MaskedMSELoss().to(device)
# optimizer = get_optimizer(model, lr = CFG.lr)
# scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)

In [22]:
# validate(model, test_dl, criterion, verbose=True)

In [23]:
# # loss_list, val_loss_list, all_loss_list
# # pred1, y1, 
# loss_list, val_loss_list, all_loss_list = train(
#     model,
#     criterion,
#     optimizer,
#     num_epochs=CFG.num_epochs,
#     train_dataloader=train_dl,
#     val_dataloader=test_dl,
#     scheduler=scheduler,
#     do_validate=True,
#     verbose=-2,
# )

In [24]:
def get_aqi_category_indices(preds, thresholds):
    indices = torch.bucketize(preds, torch.tensor(thresholds).to(device), right=False) - 1
    return torch.clamp(indices, min=0, max=5)

def compute_aqi_classification_metrics(pred, label, mask):
    """
    Compute per-class accuracy, precision, and recall separately for PM2.5 and PM10.

    Args:
        pred: Tensor of predicted AQI values of shape [N, S, 2].
        label: Tensor of ground truth AQI values of shape [N, S, 2].
        mask: Tensor of 0s and 1s indicating valid positions of shape [N, S, 1].

    Returns:
        Dictionary with separate per-class metrics for PM2.5 and PM10.
    """
    if mask.shape[-1] == 1:
        mask = mask.expand(-1, -1, 2)
    mask = mask.bool()

    metrics = {}
    for i, (name, thresholds) in enumerate(zip(CFG.target_list, [CFG.pm2_5_thresholds, CFG.pm10_thresholds])):
        pred_class = get_aqi_category_indices(pred[..., i], thresholds)
        label_class = get_aqi_category_indices(label[..., i], thresholds)

        pred_class = pred_class[mask[..., i]]
        label_class = label_class[mask[..., i]]

        class_metrics = {}
        for class_idx, class_name in enumerate(CFG.aqi_category):
            true_pos = ((pred_class == class_idx) & (label_class == class_idx)).sum().item()
            total_pred = (pred_class == class_idx).sum().item()
            total_true = (label_class == class_idx).sum().item()

            accuracy = true_pos / total_true if total_true else 0.0
            precision = true_pos / total_pred if total_pred else 0.0
            recall = true_pos / total_true if total_true else 0.0

            class_metrics[class_name] = {
                "true": true_pos,
                "total": total_true,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall
            }

        metrics[name] = class_metrics

    return metrics


In [25]:
def predict_eval(model, test_dl, criterion, verbose=True):
    o = validate(model, test_dl, criterion, verbose=verbose, get_predict=True)
    pred = torch.concat(o[0])
    y = torch.concat(o[1])
    mask = torch.concat(o[2])
    model_performance = {0 : {}, 1 : {}}
    for i, t in enumerate(CFG.target_list):
        for n, m in masked_metrics_dict.items():
            l = m(pred[:, :, i], y[:, :, i], mask[:, :, 0])
            if n == 'MSE':
                # print(y1[:, :, i].shape, pred1[:, :, i].shape, mask1.shape)
                l = np.sqrt(l.cpu().numpy())
                if verbose:
                    print(f"{t}: {n}: {l}")
            elif verbose:
                print(f"{t}: {n}: {l}")

            model_performance[i][n] = float(l)

    model_performance['classification'] = compute_aqi_classification_metrics(pred, y, mask)
    
    return model_performance

In [26]:
# predict_eval(model, test_dl, criterion, verbose=True)
# print()

In [27]:
# predict_eval(model, test_dl, criterion, verbose=True)
# print()

## Df 2

In [28]:
full_df = pd.read_csv('/kaggle/input/airdelhi-baselines-deepengineering/full_dense_df.csv')

full_df = full_df.drop(columns = 'Unnamed: 0')
full_df['date_value'] = pd.to_datetime(full_df['date_value'])

full_df = add_lag_features(full_df, lags = [7])
CFG.features = ['timeOfDay', 'lat', 'lon', 'distance', 'bus_count', 
                'day_of_week', 'pm2_5', 'pm10', 'pm2_5_lag_7', 'pm10_lag_7']
full_df = full_df.sort_values(by=['lat', 'lon', 'timeOfDay', 'date_value'])

full_df = target_scaler.transform(full_df)

full_df['day_of_week'] = full_df['date_value'].dt.dayofweek

scaler = MinMaxScaler()
full_df[CFG.features] = scaler.fit_transform(full_df[CFG.features])

full_df = rounded_lat_lon(full_df)

full_train_ds= SequenceImageDataset(full_df[full_df['date_value'] <= max_train_date], img_list)
full_test_ds = SequenceImageDataset(full_df[full_df['date_value'] > max_train_date], img_list)

full_train_dl = DataLoader(full_train_ds, batch_size = CFG.batch_size, shuffle=True)
full_test_dl = DataLoader(full_test_ds, batch_size = CFG.batch_size, shuffle=True)

<ipython-input-5-a10b91631712>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = grouped.apply(fill_na_with_previous_mean)


# Ablation Studies

In [29]:
import itertools
import json

def run_grid_search(
    param_grid: dict,
        df_dict,
        model_name,
    model_save_dir: str,
    result_json_path: str,
    default_params: dict = None
):
    from copy import deepcopy

    # Generate all parameter combinations
    keys, values = zip(*param_grid.items())
    
    param_combinations = [
        {**(default_params or {}), **dict(zip(keys, v))}
        for v in itertools.product(*values)
    ]
    
    model_dict = {}
    results_list = []
    
    for idx, param_set in enumerate(param_combinations):
        print(f"Training model {idx} with params: {param_set}")
        
        model = get_model(**param_set)
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=df_dict['train_dl'],
            val_dataloader=df_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pt"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model

        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
    
            for df_name, df1 in df_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "parameters": deepcopy(param_set),
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [30]:
param_grid = {
    # 'hidden_size' : [16, 32, 64, 128], 
    'hidden_size' : [64], 
    'num_layers' : [1, 3, 6, 9],
    'model_kind': ['rnn', 'gru']
}

default_params = {
    'input_size' : 10, 
    'forecast_horizon' : 35
}

df_dict = {
    'train_dl' : train_dl,
    'test_dl' : test_dl,
    'full_train_dl' : full_train_dl,
    'full_test_dl' : full_test_dl,
}

model_name = "model_size_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict = run_grid_search(
    param_grid,
        df_dict,
        model_name,
    model_save_dir,
    result_json_path,
    default_params
)

Training model 0 with params: {'input_size': 10, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 1, 'model_kind': 'rnn'}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
100%|██████████| 50/50 [01:16<00:00,  1.52s/it]
<ipython-input-29-c971aca37c8d>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_global

Training model 1 with params: {'input_size': 10, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 1, 'model_kind': 'gru'}


100%|██████████| 50/50 [01:17<00:00,  1.55s/it]


Training model 2 with params: {'input_size': 10, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 3, 'model_kind': 'rnn'}


...

100%|██████████| 50/50 [01:28<00:00,  1.77s/it]


Training model 6 with params: {'input_size': 10, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 9, 'model_kind': 'rnn'}


100%|██████████| 50/50 [01:26<00:00,  1.73s/it]


Training model 7 with params: {'input_size': 10, 'forecast_horizon': 35, 'hidden_size': 64, 'num_layers': 9, 'model_kind': 'gru'}


100%|██████████| 50/50 [01:38<00:00,  1.97s/it]


## Ablation 2: Input Sequence Length

In [31]:
def get_dl(df, sequence_length=35, forecast_horizon=35):
    ds = SequenceImageDataset(df, 
                              sequence_length = sequence_length, forecast_horizon = forecast_horizon,
                             img_list=img_list)
    dl = DataLoader(ds, batch_size = CFG.batch_size, shuffle=True)
    return dl

In [32]:
def run_grid_search_df(
    param_grid: dict,
        df_dict,
        model_name,
    model_save_dir: str,
    result_json_path: str,
    default_params: dict = None,
    model_params: dict = None,
):
    from copy import deepcopy

    # Generate all parameter combinations
    keys, values = zip(*param_grid.items())
    
    param_combinations = [
        {**(default_params or {}), **dict(zip(keys, v))}
        for v in itertools.product(*values)
    ]
    
    model_dict = {}
    results_list = []
    
    for idx, param_set in enumerate(param_combinations):
        print(f"Training model on dataset {idx} with params: {param_set}")

        model = get_model(**model_params)

        ds_dict = {
            k : get_dl(df = v, **param_set)
            for k, v in df_dict.items()
        }
        
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=ds_dict['train_dl'],
            val_dataloader=ds_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pth"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model
        
        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
            
            for df_name, df1 in ds_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "parameters": deepcopy(param_set),
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [33]:
param_grid = {
    'sequence_length' : [35, 70, 105, 140, 175], 
}

default_params = {
    'input_size' : 10, 
    'forecast_horizon' : 35
}

model_params = {
    'input_size': 10,
    'forecast_horizon': 35,
    'num_layers': 3,
    'hidden_size': 64,
    'model_kind': 'rnn'
}

df_dict = {
    'train_dl' : df[df['date_value'] <= max_train_date],
    'test_dl' : df[df['date_value'] > max_train_date],
    'full_train_dl' : full_df[full_df['date_value'] <= max_train_date],
    'full_test_dl' : full_df[full_df['date_value'] > max_train_date],
}

model_name = "rnn_sequence_length_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict1 = run_grid_search_df(
    param_grid,
        df_dict,
        model_name,
    model_save_dir,
    result_json_path,
    default_params=None,
    model_params = model_params,
)

Training model on dataset 0 with params: {'sequence_length': 35}


100%|██████████| 50/50 [01:18<00:00,  1.57s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with params: {'sequence_length': 70}


100%|██████████| 50/50 [01:18<00:00,  1.58s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 2 with params: {'sequence_length': 105}


100%|██████████| 50/50 [01:18<00:00,  1.58s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 3 with params: {'sequence_length': 140}


100%|██████████| 50/50 [01:18<00:00,  1.57s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 4 with params: {'sequence_length': 175}


100%|██████████| 50/50 [01:17<00:00,  1.55s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

In [34]:
model_params = {
    'input_size': 10,
    'forecast_horizon': 35,
    'num_layers': 3,
    'hidden_size': 64,
    'model_kind': 'gru'
}

df_dict = {
    'train_dl' : df[df['date_value'] <= max_train_date],
    'test_dl' : df[df['date_value'] > max_train_date],
    'full_train_dl' : full_df[full_df['date_value'] <= max_train_date],
    'full_test_dl' : full_df[full_df['date_value'] > max_train_date],
}

model_name = "gru_sequence_length_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict2 = run_grid_search_df(
    param_grid,
        df_dict,
        model_name,
    model_save_dir,
    result_json_path,
    default_params=None,
    model_params = model_params,
)

Training model on dataset 0 with params: {'sequence_length': 35}


100%|██████████| 50/50 [01:20<00:00,  1.60s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with params: {'sequence_length': 70}


100%|██████████| 50/50 [01:22<00:00,  1.64s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 2 with params: {'sequence_length': 105}


100%|██████████| 50/50 [01:23<00:00,  1.67s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 3 with params: {'sequence_length': 140}


100%|██████████| 50/50 [01:24<00:00,  1.69s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 4 with params: {'sequence_length': 175}


100%|██████████| 50/50 [01:25<00:00,  1.70s/it]
<ipython-input-32-03f43a5151d9>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

## Ablation 3: Forecast Horizon

In [35]:
def run_grid_search_horizon(
    horizon_values,
        df_dict,
        model_name,
    model_save_dir: str,
    result_json_path: str,
    df_params: dict = None,
    model_params: dict = None,
):    
    model_dict = {}
    results_list = []
    
    for idx, h in enumerate(horizon_values):
        print(f"Training model on dataset {idx} with horizon: {h}")

        model = get_model(forecast_horizon=h, **model_params)

        ds_dict = {
            k : get_dl(df = v, forecast_horizon = h, **df_params)
            for k, v in df_dict.items()
        }
        
        criterion = MaskedMSELoss().to(device)
        optimizer = get_optimizer(model, lr = CFG.lr)
        scheduler = get_reduce_lr(optimizer, factor=CFG.factor, patience=CFG.patience)
        
        loss_list, val_loss_list, all_loss_list = train(
            model,
            criterion,
            optimizer,
            num_epochs=CFG.num_epochs,
            train_dataloader=ds_dict['train_dl'],
            val_dataloader=ds_dict['test_dl'],
            scheduler=scheduler,
            do_validate=True,
            validate_frequency=1,
            verbose=-2,
            model_name=model_save_dir + f"{model_name}_{idx}_"
        )
        
        model_path = model_save_dir + f"final_{model_name}_{idx}.pt"
        torch.save(model.state_dict(), model_path)
        model_dict[idx] = model
        
        result_metrics = {}
        paths = ['best', '2_5', '10']
        for p in paths:
            result_metrics[p] = {}
            model_state_dict = torch.load(model_save_dir + f"{model_name}_{idx}_{p}.pth")
            model.load_state_dict(model_state_dict)
            
            for df_name, df1 in ds_dict.items():
                result_metrics[p][df_name] = predict_eval(model, df1, criterion, verbose=False)

        results_list.append({
            "model_index": idx,
            "model_name": model_path,
            "horizon": h,
            "params" : model_params,
            "results": result_metrics
        })

    with open(result_json_path, "w") as f:
        json.dump(results_list, f, indent=4)

    return model_dict


In [36]:
horizon_values = [35, 70, 105, 140, 175]

model_params = {
    'input_size': 10,
    'num_layers': 3,
    'hidden_size': 64,
    'model_kind': 'rnn'
}

df_dict = {
    'train_dl' : df[df['date_value'] <= max_train_date],
    'test_dl' : df[df['date_value'] > max_train_date],
    'full_train_dl' : full_df[full_df['date_value'] <= max_train_date],
    'full_test_dl' : full_df[full_df['date_value'] > max_train_date],
}

df_params = {
    'sequence_length' : 105,
}

model_name = "rnn_forecast_horizon_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict3 = run_grid_search_horizon(
    horizon_values,
        df_dict,
        model_name,
    model_save_dir,
    result_json_path,
    df_params=df_params,
    model_params = model_params,
)

Training model on dataset 0 with horizon: 35


100%|██████████| 50/50 [01:18<00:00,  1.57s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with horizon: 70


100%|██████████| 50/50 [01:17<00:00,  1.55s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 2 with horizon: 105


100%|██████████| 50/50 [01:15<00:00,  1.51s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 3 with horizon: 140


100%|██████████| 50/50 [01:13<00:00,  1.48s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 4 with horizon: 175


100%|██████████| 50/50 [01:12<00:00,  1.45s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

In [37]:
model_params = {
    'input_size': 10,
    'num_layers': 3,
    'hidden_size': 64,
    'model_kind': 'gru'
}

df_dict = {
    'train_dl' : df[df['date_value'] <= max_train_date],
    'test_dl' : df[df['date_value'] > max_train_date],
    'full_train_dl' : full_df[full_df['date_value'] <= max_train_date],
    'full_test_dl' : full_df[full_df['date_value'] > max_train_date],
}

df_params = {
    'sequence_length' : 105,
}

model_name = "gru_forecast_horizon_test"

model_save_dir = f"./{model_name}_dir/"
result_json_path = f"{model_save_dir}{model_name}_results.json"
os.makedirs(model_save_dir, exist_ok=True)

model_dict3 = run_grid_search_horizon(
    horizon_values,
        df_dict,
        model_name,
    model_save_dir,
    result_json_path,
    df_params=df_params,
    model_params = model_params,
)

Training model on dataset 0 with horizon: 35


100%|██████████| 50/50 [01:23<00:00,  1.66s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 1 with horizon: 70


100%|██████████| 50/50 [01:22<00:00,  1.65s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 2 with horizon: 105


100%|██████████| 50/50 [01:19<00:00,  1.60s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 3 with horizon: 140


100%|██████████| 50/50 [01:18<00:00,  1.58s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di

Training model on dataset 4 with horizon: 175


100%|██████████| 50/50 [01:16<00:00,  1.54s/it]
<ipython-input-35-c7319f056be7>:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(model_save_di